<a href="https://colab.research.google.com/github/himanshux06/byteuprise_ds_01/blob/main/Adult_Census_Income_Analysis_and_PredictionBYTE%20UPRISE%20TASK%202%20DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/adult-census-income/adult.csv")

In [ ]:
df.head()

In [ ]:
# Removing the useless columns that do not add any value to my analysis

In [ ]:
df.head()

In [ ]:
# Renaming the columns

In [ ]:
df.rename(columns={"education.num":"edu","marital.status":"marital","occupation":"job","hours.per.week":"hours","native.country":"country"},inplace=True)

In [ ]:
df.info()

In [ ]:
df.isna().sum()
#There are null values but they are denoted as "?" so i have to take care of that

In [ ]:
df.replace("?",pd.NA,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# There are more than 30k rows of data so i can drop the null values although i can fill them with mode form there respective columns
# but i think that will make the data incorrect

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df["age"].plot(kind="kde")

In [ ]:
df["workclass"].value_counts().plot(kind="bar")
# Most of the people have private jobs
# there is a catogery with "without-pay" they must be the unpaid interns haha
# i will drop them as they do not contribute any income

In [ ]:
df["workclass"].unique()

In [ ]:
df = df[df["workclass"] != 'Without-pay']

In [ ]:
df["workclass"].value_counts().plot(kind="bar")
# This looks better i feel sorry for thos unpaid interns

In [ ]:
df.head()

In [ ]:
df["edu"].value_counts().plot(kind="bar")
# Most of the people have HS-grade of education there are few with no education also that is denoted by 1
# refer to the education column from the actual dataset to know more about the education and there coressponding values

In [ ]:
marital_counts = df['marital'].value_counts()
plt.figure(figsize=(10,6))
patches, texts, _ = plt.pie(marital_counts, autopct='%1.1f%%', startangle=140)
plt.title('Marital Status Distribution')
plt.legend(patches, marital_counts.index, title="Marital Status", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.show()

In [ ]:
df["job"].value_counts()

In [ ]:
# I think i can just use One-Hot encoding for this column as there not many Categories

In [ ]:
df.job.value_counts().plot(kind="bar")

In [ ]:
df.head()

In [ ]:
df["relationship"].value_counts().plot(kind="bar")

In [ ]:
# Checking relationship between work class and income
df.groupby("workclass").income.value_counts().plot(kind="bar")
# Most of the people have private earn less then 50k

In [ ]:
# education vs income

In [ ]:
df.groupby("income").edu.mean().reset_index().sort_values(by="income")
# Those who earn more than 50k have better education so kids study hard

In [ ]:
df.head()

In [ ]:
df["race"].value_counts()

In [ ]:
df.race.value_counts().plot(kind="pie")
# most of the peple are white followed by Black and Asians

In [ ]:
# Race vs marital status

In [ ]:
df.groupby("marital").race.value_counts().plot(kind="bar")

In [ ]:
# Hours vs job type

In [ ]:
df.groupby("job").hours.mean().reset_index().sort_values(by="hours")
# Farming and fishing workers work the most number of hours
# private house serv type job have the least number of hours

In [ ]:
df.head()

In [ ]:
df.income.value_counts().plot(kind="bar")
# most of the people earn less than 50k

In [ ]:
# Model building and evaluation

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
# Encoding

In [ ]:
# work class i am using frequency encoding
location_freq = df['workclass'].value_counts(normalize=True).to_dict()
df['workclass_freq'] = df['workclass'].map(location_freq)

In [ ]:
df.head()

In [ ]:
# marital i am using one hot encoding as there are less number of unqiue of values
df['marital'] = df['marital'].astype('category')
df = pd.get_dummies(df, columns=['marital'], prefix='marital')

In [ ]:
df.head()

In [ ]:
# job -  i am using frequency encoding
location_freq = df['job'].value_counts(normalize=True).to_dict()
df['Job'] = df['job'].map(location_freq)

In [ ]:
df.drop(columns=["job"],inplace=True)

In [ ]:
df.head()

In [ ]:
# relationship -  i am using one hot encoding as there are less number of unqiue of values
df['relationship'] = df['relationship'].astype('category')
df = pd.get_dummies(df, columns=['relationship'], prefix='relationship')

In [ ]:
df.head()

In [ ]:
# country - i am using one hot encoding as there are less number of unqiue of values
df['country'] = df['country'].astype('category')
df = pd.get_dummies(df, columns=['country'], prefix='country')

In [ ]:
df['Income'] = df['income'].apply(lambda x: 0 if x == '<=50K' else 1)

In [ ]:
df.drop(columns=["income"],inplace=True)

In [ ]:
df['race'] = df['race'].astype('category')
df = pd.get_dummies(df, columns=['race'], prefix='race')

In [ ]:
df['sex'] = df['sex'].astype('category')
df = pd.get_dummies(df, columns=['sex'], prefix='sex')

In [ ]:
df.head()

In [ ]:
# normalizing hours and age column using min - max scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
age_values = df['age'].values.reshape(-1, 1)
hours_values = df['hours'].values.reshape(-1, 1)
df['age_normalized'] = scaler.fit_transform(age_values)
df['hours_normalized'] = scaler.fit_transform(hours_values)

In [ ]:
df.drop(columns=["age","hours"],inplace=True)

In [ ]:
df["Income"]=df["Income"].astype("int")
df["edu"]=df["edu"].astype("int")

In [ ]:
X = df.drop(columns=["Income"])
y = df['Income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=int(max(y)))

# I have used GridSearchCV on my local machine and best parameters obtained from GridSearchCV are listed below
best_params = {
    'n_estimators': 300,
    'max_depth': 20,
    'min_samples_split': 10,
    'min_samples_leaf': 1
}

# Initialize the RandomForestClassifier with the best parameters
best_rf_classifier = RandomForestClassifier(**best_params)

# Train the classifier
best_rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = best_rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nCddonfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# The recall of 0 that is <=50 is 92% which is really good because we had more values for the 0 value

In [ ]:
# to improve this we can use SMOTE technique